In [13]:
from gurobipy import Model, quicksum, GRB

In [14]:
Plnt = {'plnt'}
DC = set(range(1,4))
Cust = DC.copy()

In [28]:
del_cost = {(j, k):10 for j in DC for k in Cust}
del_cost

{(1, 1): 10,
 (1, 2): 10,
 (1, 3): 10,
 (2, 1): 10,
 (2, 2): 10,
 (2, 3): 10,
 (3, 1): 10,
 (3, 2): 10,
 (3, 3): 10}

In [29]:
demand = {k:10 for k in Cust}
demand

{1: 10, 2: 10, 3: 10}

In [20]:
model = Model()

In [22]:
x = {}

for j in DC:
    for k in Cust:
        x[j,k] = model.addVar(vtype="C", name=f'x[{j},{k}]')

model.update()

In [ ]:
Cust_Demand_Cons = {}

for k in Cust:
    Cust_Demand_Cons[k] = model.addCo

In [24]:
model.setObjective(
    quicksum(del_cost[j,k] * x[j,k] for j in DC for k in Cust)
    ,GRB.MINIMIZE
)
model.update()

In [25]:
model.optimize()

Optimize a model with 0 rows, 18 columns and 0 nonzeros
Coefficient statistics:
  Matrix range     [0e+00, 0e+00]
  Objective range  [1e+01, 1e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [0e+00, 0e+00]
Presolve removed 0 rows and 18 columns
Presolve time: 0.01s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.02 seconds
Optimal objective  0.000000000e+00


In [26]:
for v in model.getVars():
    print(v.VarName, v.X)

x[1,1] 0.0
x[1,2] 0.0
x[1,3] 0.0
x[2,1] 0.0
x[2,2] 0.0
x[2,3] 0.0
x[3,1] 0.0
x[3,2] 0.0
x[3,3] 0.0
x[1,1] 0.0
x[1,2] 0.0
x[1,3] 0.0
x[2,1] 0.0
x[2,2] 0.0
x[2,3] 0.0
x[3,1] 0.0
x[3,2] 0.0
x[3,3] 0.0
